# Foundry IQ: Grounding with Azure AI Search

> **Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub
> **Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)
> **Copyright 2025 Ozgur Guler. All rights reserved.**

---

## What is Foundry IQ?

**Foundry IQ** is Azure AI Foundry's intelligent retrieval system that connects AI agents to your proprietary data. It enables **Retrieval-Augmented Generation (RAG)** by automatically:

1. **Intercepting user queries** before they reach the LLM
2. **Searching your indexed content** in Azure AI Search
3. **Injecting relevant context** into the LLM prompt
4. **Generating grounded responses** based on your actual data

### Why Grounding Matters

Without grounding, LLMs can only use their training data (which has a knowledge cutoff) and may "hallucinate" - generating plausible but incorrect information. Grounding solves this by:

| Problem | Solution with Grounding |
|---------|------------------------|
| Knowledge cutoff | Access to your latest indexed documents |
| Hallucination | Responses anchored to real retrieved content |
| Generic answers | Domain-specific responses from your data |
| No citations | Source attribution from search results |

---

## Architecture: What Happens Under the Hood

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                              USER QUERY                                      │
│                    "What is the IMF's economic outlook?"                     │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                         AZURE AI FOUNDRY AGENT                               │
│  ┌─────────────────────────────────────────────────────────────────────┐    │
│  │  1. Agent receives query                                             │    │
│  │  2. Recognizes AzureAISearchTool is available                       │    │
│  │  3. Decides to call the tool (based on instructions)                │    │
│  └─────────────────────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                      AZURE AI SEARCH (Foundry IQ)                            │
│  ┌─────────────────────────────────────────────────────────────────────┐    │
│  │  4. Query is sent to AI Search index                                │    │
│  │  5. Search executes (SIMPLE, SEMANTIC, or VECTOR)                   │    │
│  │  6. Top-K relevant documents retrieved                              │    │
│  │  7. Results returned with relevance scores                          │    │
│  └─────────────────────────────────────────────────────────────────────┘    │
│                                                                              │
│  Index: imf_baseline                                                         │
│  ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐                        │
│  │ Doc 1    │ │ Doc 2    │ │ Doc 3    │ │ Doc N    │                        │
│  │ Score:   │ │ Score:   │ │ Score:   │ │ Score:   │                        │
│  │ 0.95     │ │ 0.87     │ │ 0.82     │ │ ...      │                        │
│  └──────────┘ └──────────┘ └──────────┘ └──────────┘                        │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                              LLM (gpt-5-nano)                                │
│  ┌─────────────────────────────────────────────────────────────────────┐    │
│  │  8. LLM receives:                                                    │    │
│  │     - Original user query                                           │    │
│  │     - Retrieved document chunks (grounding context)                 │    │
│  │     - Agent instructions (cite sources, don't hallucinate)          │    │
│  │                                                                      │    │
│  │  9. LLM generates response grounded in retrieved content            │    │
│  └─────────────────────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                           GROUNDED RESPONSE                                  │
│  "According to the IMF World Economic Outlook [Doc 1], the global           │
│   economy is projected to grow at 3.2% in 2024..."                          │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## Configuration

We'll connect to the following AI Search index:
- **AI Search Service**: `chatops`
- **Index Name**: `imf_baseline`

## Prerequisites

1. **Azure CLI authenticated**: Run `az login`
2. **Azure AI Foundry project**: From previous sections
3. **Azure AI Search service**: With an existing index
4. **Connection configured**: Between Foundry project and AI Search service

---

## Section 1: Setup and Configuration

In [ ]:
# Install required packages
!pip install azure-ai-projects --pre --quiet
!pip install azure-identity python-dotenv requests --quiet

print("Packages installed")

In [ ]:
import os
from dotenv import load_dotenv

# Load environment from parent directory
load_dotenv("../.env")

# Foundry Configuration
FOUNDRY_ACCOUNT = os.getenv("FOUNDRY_ACCOUNT_NAME", "ozgurguler-7212-resource")
PROJECT_NAME = os.getenv("FOUNDRY_PROJECT_NAME", "ozgurguler-7212")
PROJECT_ENDPOINT = f"https://{FOUNDRY_ACCOUNT}.services.ai.azure.com/api/projects/{PROJECT_NAME}"

# AI Search Configuration
AI_SEARCH_SERVICE = os.getenv("AI_SEARCH_SERVICE", "chatops")
AI_SEARCH_ENDPOINT = f"https://{AI_SEARCH_SERVICE}.search.windows.net"
AI_SEARCH_INDEX = os.getenv("AI_SEARCH_INDEX", "imf_baseline")

# Model Configuration
CHAT_MODEL = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-5-nano")

# Agent Configuration
AGENT_NAME = "imf-grounded-agent"

print(f"Project Endpoint: {PROJECT_ENDPOINT}")
print(f"AI Search Endpoint: {AI_SEARCH_ENDPOINT}")
print(f"AI Search Index: {AI_SEARCH_INDEX}")
print(f"Model: {CHAT_MODEL}")

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# Initialize the client
credential = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

print("AIProjectClient initialized successfully")

---

## Section 2: Verify AI Search Connection

First, let's check if a connection to the AI Search service exists in the project.

In [ ]:
# Skip connection listing - go directly to getting the default AI Search connection
print("Skipping connection list (can be slow)...")
print("Will get default AI Search connection in next cell.")

In [ ]:
from azure.ai.projects.models import ConnectionType

# Try to get the default AI Search connection
print("Getting default AI Search connection...")

try:
    ai_search_connection = client.connections.get_default(ConnectionType.AZURE_AI_SEARCH)
    AI_SEARCH_CONNECTION_ID = ai_search_connection.id
    
    print(f"\nDefault AI Search connection found!")
    print(f"  Name: {ai_search_connection.name}")
    print(f"  ID: {ai_search_connection.id}")
    
except Exception as e:
    print(f"\nNo default AI Search connection: {e}")
    print("\nUsing manual connection ID from earlier output...")
    
    # Use the connection ID we saw earlier in the output
    AI_SEARCH_CONNECTION_ID = "/subscriptions/a20bc194-9787-44ee-9c7f-7c3130e651b6/resourceGroups/rg-ozgurguler-7212/providers/Microsoft.CognitiveServices/accounts/ozgurguler-7212-resource/projects/ozgurguler-7212/connections/chatopsozgulert2mx2h"
    print(f"  Using: {AI_SEARCH_CONNECTION_ID}")

### Section 2b: Create AI Search Connection (if needed)

If no connection exists, create one using the Azure ML SDK.

In [ ]:
# Create AI Search connection if not found
CREATE_CONNECTION = False  # Set to True to create

if CREATE_CONNECTION and AI_SEARCH_CONNECTION_ID is None:
    print("Creating AI Search connection...")
    
    try:
        from azure.ai.ml import MLClient
        from azure.ai.ml.entities import AzureAISearchConnection
        
        # Get ML client
        ml_client = MLClient(
            credential=credential,
            subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
            resource_group_name=os.getenv("AZURE_RESOURCE_GROUP", "rg-ozgurguler-7212"),
            workspace_name=PROJECT_NAME
        )
        
        # Create connection (using AAD auth - no API key)
        connection = AzureAISearchConnection(
            name=f"{AI_SEARCH_SERVICE}-connection",
            endpoint=AI_SEARCH_ENDPOINT,
            api_key=None  # Use AAD authentication
        )
        
        ml_client.connections.create_or_update(connection)
        print(f"✅ Created connection: {connection.name}")
        
        # Refresh connection ID
        ai_search_connection = client.connections.get_default(ConnectionType.AZURE_AI_SEARCH)
        AI_SEARCH_CONNECTION_ID = ai_search_connection.id
        
    except Exception as e:
        print(f"❌ Error creating connection: {e}")
        print("\nAlternatively, create the connection via Azure Portal:")
        print("1. Go to Azure AI Foundry portal")
        print("2. Select your project")
        print("3. Go to Settings > Connections")
        print("4. Add a new Azure AI Search connection")
else:
    if AI_SEARCH_CONNECTION_ID:
        print("AI Search connection already exists")
    else:
        print("Connection creation skipped (CREATE_CONNECTION = False)")

---

## Section 3: Verify the Search Index

Let's verify the `imf_baseline` index exists and check its schema.

In [ ]:
import subprocess
import json

# Get AI Search service info
print(f"Checking index '{AI_SEARCH_INDEX}' on '{AI_SEARCH_SERVICE}'...\n")

# Use REST API to check index (requires proper auth)
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import requests

try:
    # Get token for AI Search
    token_provider = get_bearer_token_provider(
        DefaultAzureCredential(),
        "https://search.azure.com/.default"
    )
    token = token_provider()
    
    # Check index exists
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(
        f"{AI_SEARCH_ENDPOINT}/indexes/{AI_SEARCH_INDEX}?api-version=2024-07-01",
        headers=headers
    )
    
    if response.status_code == 200:
        index_info = response.json()
        print(f"✅ Index '{AI_SEARCH_INDEX}' found!\n")
        print(f"Fields:")
        for field in index_info.get('fields', [])[:10]:  # Show first 10 fields
            print(f"  - {field['name']} ({field['type']})")
        if len(index_info.get('fields', [])) > 10:
            print(f"  ... and {len(index_info['fields']) - 10} more fields")
    else:
        print(f"⚠️  Index not found or access denied: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"Error checking index: {e}")
    print("\nNote: You may need proper RBAC permissions on the AI Search service.")

---

## Section 4: Configure the Azure AI Search Tool

### How the Tool Works

The `AzureAISearchAgentTool` is a **function-calling tool** that the LLM can invoke during a conversation. When configured:

1. **Tool Registration**: The tool definition is sent to the LLM as part of the system context
2. **Tool Invocation**: When the LLM decides it needs information, it generates a tool call
3. **Search Execution**: Foundry executes the search against your AI Search index
4. **Result Injection**: Retrieved documents are added to the conversation context
5. **Response Generation**: The LLM uses the retrieved content to formulate a response

### Query Types Explained

| Query Type | How It Works | Best For |
|------------|--------------|----------|
| `SIMPLE` | Keyword matching with BM25 ranking | Exact term searches, structured queries |
| `SEMANTIC` | AI-powered understanding of meaning | Natural language questions, concept matching |
| `VECTOR` | Embedding similarity search | Finding conceptually similar content |
| `VECTOR_SIMPLE_HYBRID` | Combines keyword + vector | Balanced precision and recall |
| `VECTOR_SEMANTIC_HYBRID` | Combines semantic + vector | Best overall relevance |

### Tool Structure

```python
AzureAISearchAgentTool(
    azure_ai_search=AzureAISearchToolResource(
        indexes=[
            AISearchIndexResource(
                project_connection_id="...",  # Connection to AI Search service
                index_name="...",              # Which index to search
                query_type="SIMPLE",           # How to search
            )
        ]
    )
)
```

In [ ]:
from azure.ai.projects.models import (
    AzureAISearchAgentTool,
    AzureAISearchToolResource,
    AISearchIndexResource,
    AzureAISearchQueryType,
)

# Configure the Azure AI Search tool
print("Configuring Azure AI Search tool...\n")

if AI_SEARCH_CONNECTION_ID:
    # Create the AI Search tool with the correct nested structure
    ai_search_tool = AzureAISearchAgentTool(
        azure_ai_search=AzureAISearchToolResource(
            indexes=[
                AISearchIndexResource(
                    project_connection_id=AI_SEARCH_CONNECTION_ID,
                    index_name=AI_SEARCH_INDEX,
                    query_type=AzureAISearchQueryType.SIMPLE,
                )
            ]
        )
    )
    
    print(f"AI Search tool configured:")
    print(f"  Connection ID: {AI_SEARCH_CONNECTION_ID}")
    print(f"  Index: {AI_SEARCH_INDEX}")
    print(f"  Query Type: SIMPLE")
else:
    print("Cannot configure tool - no AI Search connection ID")
    print("Please create an AI Search connection first (Section 2b)")
    ai_search_tool = None

---

## Section 5: Create Grounded Agent

### Agent Definition Components

When we create a grounded agent, we're combining several elements:

```
┌─────────────────────────────────────────────────────────────────┐
│                    PromptAgentDefinition                         │
├─────────────────────────────────────────────────────────────────┤
│  model: "gpt-5-nano"                                            │
│    └─ Which LLM processes queries and generates responses       │
│                                                                  │
│  instructions: "You are a helpful assistant..."                 │
│    └─ System prompt that guides agent behavior                  │
│    └─ CRITICAL: Include grounding rules here!                   │
│                                                                  │
│  tools: [AzureAISearchAgentTool(...)]                           │
│    └─ Available tools the agent can invoke                      │
│    └─ Agent decides WHEN to use tools based on query            │
└─────────────────────────────────────────────────────────────────┘
```

### The Importance of Instructions

The agent instructions are crucial for effective grounding. They tell the LLM:

1. **When to search**: "You MUST use the Azure AI Search tool..."
2. **How to use results**: "Ground your answers in the retrieved documents"
3. **How to handle failures**: "If search returns no results, say..."
4. **Citation requirements**: "Always cite your sources..."

Without proper instructions, the LLM might:
- Answer from its training data instead of searching
- Ignore retrieved content
- Fail to cite sources
- Hallucinate when search returns no results

In [ ]:
# Agent instructions optimized for grounding
GROUNDED_AGENT_INSTRUCTIONS = f"""
You are a helpful assistant that answers questions using the IMF (International Monetary Fund) knowledge base.

IMPORTANT RULES:
1. You MUST use the Azure AI Search tool to find relevant information before answering.
2. You MUST ground your answers in the retrieved documents.
3. If the search returns no relevant results, say "I couldn't find information about that in the knowledge base."
4. NEVER make up information - only use what you find in the search results.
5. Always cite your sources by mentioning which document the information came from.

The knowledge base contains: {AI_SEARCH_INDEX}

When responding:
- Be concise and accurate
- Quote relevant passages when helpful
- If asked about topics outside the knowledge base, politely redirect to what you can help with
"""

print("Agent Instructions:")
print(GROUNDED_AGENT_INSTRUCTIONS)

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition

# Create the grounded agent
print(f"Creating grounded agent: {AGENT_NAME}...\n")

if ai_search_tool:
    try:
        # Check if agent already exists
        try:
            existing_agent = client.agents.get(agent_name=AGENT_NAME)
            print(f"Agent already exists: {existing_agent.name} (version: {existing_agent.version})")
            agent = existing_agent
        except:
            # Create new agent using PromptAgentDefinition
            agent = client.agents.create_version(
                agent_name=AGENT_NAME,
                definition=PromptAgentDefinition(
                    model=CHAT_MODEL,
                    instructions=GROUNDED_AGENT_INSTRUCTIONS,
                    tools=[ai_search_tool],  # Pass the AzureAISearchAgentTool
                )
            )
            print(f"Created grounded agent: {agent.name}")
            print(f"   Version: {agent.version}")
            print(f"   Model: {CHAT_MODEL}")
        
    except Exception as e:
        print(f"Error creating agent: {e}")
        import traceback
        traceback.print_exc()
        agent = None
else:
    print("Cannot create agent - AI Search tool not configured")
    agent = None

---

## Section 6: Test the Grounded Agent

### What Happens During Agent Invocation

When you send a query to the grounded agent, here's the detailed flow:

```
Step 1: User sends query
        "What is the IMF's role in global economic stability?"
                              │
                              ▼
Step 2: Agent analyzes query + available tools
        - Sees AzureAISearchTool is available
        - Instructions say "MUST use search tool"
        - Decides to invoke the tool
                              │
                              ▼
Step 3: Tool call generated (internal)
        {
          "tool": "azure_ai_search",
          "query": "IMF role global economic stability",
          "index": "imf_baseline"
        }
                              │
                              ▼
Step 4: Foundry executes search
        - Connects to AI Search via connection ID
        - Runs query against imf_baseline index
        - Retrieves top-K documents (default: 5)
                              │
                              ▼
Step 5: Results injected into context
        [Retrieved Document 1]: "The IMF promotes international..."
        [Retrieved Document 2]: "Key functions include surveillance..."
        [Retrieved Document 3]: "The IMF provides financial assistance..."
                              │
                              ▼
Step 6: LLM generates grounded response
        - Uses retrieved documents as primary source
        - Follows instructions to cite sources
        - Avoids hallucination by sticking to retrieved content
                              │
                              ▼
Step 7: Response returned to user
        "According to the IMF documentation, the organization plays
         several key roles in global economic stability: [1] promoting
         international monetary cooperation, [2] providing surveillance
         of economic policies..."
```

### Understanding the Response

The agent's response should:
- **Reference retrieved content**: Quotes or paraphrases from indexed documents
- **Include citations**: Indicates which documents provided the information
- **Acknowledge limitations**: Says "I couldn't find..." if search returns nothing
- **Stay on topic**: Redirects questions outside the knowledge base scope

In [ ]:
# Get OpenAI client for agent invocation
openai_client = client.get_openai_client()

def ask_grounded_agent(question: str, agent_name: str) -> str:
    """Send a question to the grounded agent and get a response."""
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print("="*60)
    
    try:
        # Create a conversation
        conversation = openai_client.conversations.create()
        
        # Send the message with agent reference
        response = openai_client.responses.create(
            input=question,
            conversation=conversation.id,
            extra_body={"agent": {"name": agent_name, "type": "agent_reference"}},
        )
        
        print(f"\nStatus: {response.status}")
        print(f"\nAgent Response:\n{response.output_text}")
        return response.output_text
        
    except Exception as e:
        print(f"\nError: {e}")
        return None

In [ ]:
# Test questions for the IMF knowledge base
if agent:
    test_questions = [
        "What is the IMF's role in global economic stability?",
        "Summarize the key points from the latest available IMF report.",
        "What recommendations does the IMF provide for developing economies?",
    ]
    
    for question in test_questions:
        response = ask_grounded_agent(question, agent.name)
        print("\n")
else:
    print("Agent not available - please create it first")

---

## Section 7: Alternative - Using Prompt Agent with AI Search

You can also use the newer `PromptAgentDefinition` with the AI Search tool.

In [ ]:
# This section is now redundant since we're using the correct API above
# The agent was already created in cell-16 with the proper AzureAISearchAgentTool

print("Agent already created in Section 5 above")
print(f"Agent: {agent.name if agent else 'Not created'}")

In [ ]:
# Test the prompt agent using conversations API
if prompt_agent:
    openai_client = client.get_openai_client()
    
    # Create conversation
    conversation = openai_client.conversations.create()
    print(f"Created conversation: {conversation.id}\n")
    
    # Test query
    test_query = "What are the main economic indicators discussed in the IMF baseline?"
    print(f"Query: {test_query}")
    
    response = openai_client.responses.create(
        input=test_query,
        conversation=conversation.id,
        extra_body={"agent": {"name": prompt_agent.name, "type": "agent_reference"}},
    )
    
    print(f"\nStatus: {response.status}")
    print(f"\nAgent Response:\n{response.output_text}")
else:
    print("Prompt agent not available")

---

## Section 8: Cleanup (Optional)

In [ ]:
# Delete agents
DELETE_AGENTS = False  # Set to True to delete

if DELETE_AGENTS:
    try:
        if agent:
            client.agents.delete_agent(agent.id)
            print(f"Deleted agent: {agent.id}")
    except Exception as e:
        print(f"Error deleting agent: {e}")
    
    try:
        if prompt_agent:
            client.agents.delete(agent_name=PROMPT_AGENT_NAME)
            print(f"Deleted prompt agent: {PROMPT_AGENT_NAME}")
    except Exception as e:
        print(f"Error deleting prompt agent: {e}")
else:
    print("Agent deletion skipped (DELETE_AGENTS = False)")

---

## Summary

### What is Foundry IQ Doing?

**Foundry IQ** orchestrates the entire RAG (Retrieval-Augmented Generation) pipeline:

```
┌────────────────────────────────────────────────────────────────────────────┐
│                        FOUNDRY IQ ORCHESTRATION                             │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   ┌─────────┐    ┌──────────────┐    ┌─────────────┐    ┌─────────────┐   │
│   │  User   │───▶│    Agent     │───▶│  AI Search  │───▶│     LLM     │   │
│   │  Query  │    │  (Tool Call) │    │  (Retrieve) │    │  (Generate) │   │
│   └─────────┘    └──────────────┘    └─────────────┘    └─────────────┘   │
│                                                                             │
│   Key Operations:                                                           │
│   • Query Understanding    - Agent decides what to search for              │
│   • Search Execution       - Foundry IQ runs the search                    │
│   • Context Augmentation   - Results injected into LLM context             │
│   • Response Generation    - LLM produces grounded answer                  │
│   • Citation Tracking      - Source attribution maintained                 │
│                                                                             │
└────────────────────────────────────────────────────────────────────────────┘
```

### What We Built

1. **Connected to AI Search** - Established secure connection via `project_connection_id`
2. **Configured Search Tool** - Set up `AzureAISearchAgentTool` with query parameters
3. **Created Grounded Agent** - Combined LLM + tool + instructions
4. **Tested RAG Pipeline** - Verified end-to-end grounding with real queries

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Grounding** | Anchoring LLM responses in retrieved content from your data |
| **RAG** | Retrieval-Augmented Generation - search then generate pattern |
| **Tool Calling** | LLM decides when to invoke search based on query |
| **Connection** | Secure link between Foundry project and AI Search service |
| **Query Type** | Search algorithm: SIMPLE (keyword), SEMANTIC (AI), VECTOR (embeddings) |
| **Top-K** | Number of documents retrieved per search (default: 5) |

### Under the Hood: Data Flow

| Stage | Component | What Happens |
|-------|-----------|--------------|
| 1. Input | User Query | Natural language question received |
| 2. Planning | Agent/LLM | Decides to use search tool |
| 3. Retrieval | AI Search | Executes query, returns ranked documents |
| 4. Augmentation | Foundry IQ | Injects documents into LLM context |
| 5. Generation | LLM | Produces response using retrieved content |
| 6. Output | Response | Grounded answer with citations |

### Best Practices

| Practice | Why It Matters |
|----------|---------------|
| Use SEMANTIC query type | Better understanding of natural language |
| Set low temperature (0.1-0.3) | More factual, less creative responses |
| Include citation instructions | Traceability and trust |
| Add "I don't know" fallback | Prevents hallucination on empty results |
| One index per agent | Simpler reasoning, better focus |

### Troubleshooting

| Issue | Likely Cause | Solution |
|-------|--------------|----------|
| No connection found | Missing project connection | Create via Portal or SDK |
| Empty responses | Index has no matching content | Check index data, broaden query |
| Hallucination | Weak instructions | Add stricter grounding rules |
| Wrong citations | Field mapping issues | Verify index schema |
| Slow responses | Large index, complex queries | Use filters, optimize index |

---

## Next Steps

Continue to `../07-logic-apps-as-mcp-server` for integration with Logic Apps.

---

<div align="center">

## License & Attribution

This notebook is part of the **Azure AI Foundry Demo Repository**

[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](../LICENSE)

**Original Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub

**Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)

---

*If you use, modify, or distribute this work, you must provide appropriate credit to the original author as required by the [Apache License 2.0](../LICENSE).*

**Copyright © 2025 Ozgur Guler. All rights reserved.**

</div>